<a href="https://colab.research.google.com/github/Elimeleth/aprende_a_crear_asistentes/blob/main/RAG_Rerank_Groq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q langchain langchain-community langchain-core langchain-groq langchain-experimental fastembed flashrank faiss_cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 4.7 MB/s eta 0:00:00
   ━

# Comenzemos!

Para este caso usamos groq pero si deseas usar openai solo instala:

```bash
pip install --upgrade langchain-openai
```

```python

llm = ChatOpenAI(model="", temperature=.7, api_key="")
```

In [ ]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_groq import ChatGroq
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.vectorstores import FAISS

from langchain_community.document_loaders import TextLoader
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainFilter, FlashrankRerank, DocumentCompressorPipeline
from langchain_core.prompts import ChatPromptTemplate

# ChatPromptTemplate

Es la manera de crear prompts de manera estructurada en langchain para modelos de lenguajes tipo Chat, (Openai, Gemini, Llama ...)

tiene algunos conceptos avanzados, pero al caso.

system: Son las instrucciones que recibe el modelo.
placeholder: Son un set de mensajes como lo hemos venido viendo user/assistant
human: es el input que ingresa el usuario o cliente.

piensa en la estructura del prompt como un motor de plantilla, siendo las variables las que estan estre llaves: `{variable}`, para escapar una variable seria tipo: `{{esto no es una variable}}`

## Embeddings

Son modelos entrenados con cierto rango de vocabulario, su tarea es computar palabras a tokens, de esta forma se pueden calcular las distancias siguiendo formulas matematicas.

In [ ]:
llm = ChatGroq(model='llama3-70b-8192', temperature=.7, api_key="<TU-API-KEY>")
embeddings = FastEmbedEmbeddings()


prompt = ChatPromptTemplate.from_messages((
    ("system", """Answer any use questions based solely on the context below:

<context>

{context}

</context>
"""),
    ("placeholder", "{chat_history}"),
    ("human", "{input}")
))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

# Cargadores

Un cargador de texto lo que hace es extraer el texto de un archivo en fragmentos de tamaño ajustable, llamado chunks, el splitter se encarga de picar esos chunks en fragmentos con cierto tipo de semantica o sentido uno del otro.

In [ ]:
loader = TextLoader("data.txt")
splitter = SemanticChunker(embeddings, breakpoint_threshold_type="gradient")
documents = loader.load_and_split(splitter)

# Retrievers

El core de nuestra solución RAG, se encuentra en que tan bueno son nuestros modelos que extraen documentos.

nuestros documentos son esos fragmentos que cargamos anteriormente llamado chunks en este caso vienen en este formato:
```python
Document(pageContent="...", metadata={})
```

En nuestro retrieval hacemos algo llamado Rerank, el cual es una solucion avanzada que le da un toque de presicion a nuestro modelo de recuperacion de texto.

LLMChainFilter, realiza un filtrado de los documentos extraidos, algo parecido a lo que hace la metrica: `Context Relevancy Score`

Rerank, como su nombre indica reordena nuestros documentos para no sobrecargar nuestro modelo de un contexto irrelevante a la pregunta del usuario.

In [ ]:
base_retriever = FAISS.from_documents(documents, embeddings).as_retriever(search_kwargs={ "k": 5 })
compression_retriever = ContextualCompressionRetriever(
    base_compressor=DocumentCompressorPipeline(transformers=[
            LLMChainFilter.from_llm(llm),
            FlashrankRerank()
        ]), base_retriever=base_retriever
)

In [ ]:
compression_retriever.invoke("precio brunch")

[Document(metadata={'id': 2, 'relevance_score': 0.9918487, 'source': 'data.txt'}, page_content='Inicias desayunando en nuestras terrazas Los Olivos o bajo la sombra de un mezquite. Hay buffet y los ¡menores de 13 años no pagan! Y luego haces una visita guiada por todo el Rancho conociendo a nuestra comunidad animal y nuestra forma de trabajar sin dañar el medio ambiente. Después de visitar el viñedo, la quesería y la cava de maduración te ofrecemos una degustación guiada de nuestros quesos con una copa de vino de la región. Días de operación: Sábado y Domingo Brunch 10 a 13 hrs; Experiencia Rancho Sta Marina: 13 hrs\nPrecio : $580 pesos* \nNiños entre los 5 y los 12 años solo pagan 150 pesos por la Experiencia Rancho Sta Marina. *No incluye bebidas preparadas, ni alcohol ni propinas\nEste evento solo se puede reservar o cotizar con un Asesor de *Rancho Santa Marina*\n*No incluye bebidas preparadas, ni alcohol ni propinas\n\n\nDaypass Gastronómico: \nConoce Rancho Santa Marina, vive una

In [ ]:
stuff = create_stuff_documents_chain(llm, prompt)
qa_chain = create_retrieval_chain(compression_retriever, stuff)

In [ ]:
result = qa_chain.invoke({"input": "los menores pagan el daypass verano?"})

In [ ]:
"""
result contiene 3 campos:
input: la pregunta del cliente
answer: la respuesta generada por el llm
context: el contexto usado en la inferencia

result["input"], result["answer"], result["context"]
"""

result

{'input': 'los menores pagan el daypass verano?',
 'context': [Document(metadata={'id': 0, 'relevance_score': 0.9984349, 'source': 'data.txt'}, page_content='Miércoles y Jueves solo para grupos con reservación previa. Duración de la experiencia: 5 horas. Precio adulto: +14 años $800.00 MN. Precio menor: a partir de 5 años $350.00 MN. *Actividad no recomendada para menores de 5 años. enlace:https://ranchosantamarina.com/producto/day-pass-pizza/\n\n[PROMOCIONES ACTIVAS EN EL DAYPASS VERANO, INCLUYE PIZZA] \n¡Reserva para estas vacaciones el Daypass de Verano! Disfruta todo Julio y Agosto los miércoles, jueves y viernes del taller de pizza familiar con un precio especial. Incluye visita guiada con actividades en el huerto y convivencia con los animales, no olvides que los niños siempre tendrán que ir acompañados de un adulto. El menú del taller incluye: 1 pizza, ensalada, postre y agua del día por persona. $400 pesos por persona incluido menores\nEl precio se paga por persona, menor y aco